# Data Visualisation

In [ ]:
# Change directory to keep paths consistent
%cd /Users/brk/projects/masters/SU/ergo/src

### Imports and setup

In [ ]:
# minimise me
%load_ext autoreload
%autoreload 2
import seaborn as sns
import seaborn.objects as so
import matplotlib.pyplot as plt
import ipywidgets as widgets
import datetime
from ipywidgets import interact, interactive, fixed, interact_manual
import pandas as pd
import numpy as np
import models
import vis
import common
import read
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
import sklearn
import tqdm
import logging as l
import tqdm
import yaml
import glob
from matplotlib.colors import LogNorm
import re
from sklearn.metrics import classification_report
from scipy.stats import f

### Utility functions

In [ ]:
# minimise me
def heatmap(fn, X_val, y_val, axis=1):
    y_val_pred = np.argmax(tf.nn.softmax(fn(X_val)).numpy(), axis=axis)

    cm_val = tf.math.confusion_matrix(
        y_val.flatten(), 
        y_val_pred.flatten()
    ).numpy()
    cm_val[-1, -1] = 0
    return sns.heatmap(
        cm_val,
        annot=False,
        fmt='d',
        square=True,
        mask=(cm_val==0),
        cmap='viridis'
    )

def prettify_col_name(x):
    return x.split('.')[-1].replace('_', ' ').title()

def calculate_prediction_ellipse(x, y, alpha=0.5):
    data = np.column_stack((x, y)) # Combine x and y into a single data array
    num_dimensions = data.shape[1]
    num_data_points = data.shape[0]
    # Estimate the sample covariance matrix
    sample_covariance_matrix = np.cov(data, rowvar=False)
    # Calculate the sample mean for each dimension
    sample_mean = np.mean(data, axis=0)
    # Generate angles for the ellipse
    theta = np.linspace(0, 2*np.pi, num=100)
    # Calculate the radius of the ellipse. `f.ppf` is the inverse of the CDF
    radius = np.sqrt(
        num_dimensions * (num_data_points - 1) / (num_data_points - num_dimensions) *
        (1 + 1/num_data_points) * f.ppf(1 - alpha, num_dimensions, num_data_points - num_dimensions)
    )
#     print(sample_covariance_matrix)
    # Compute the Cholesky decomposition of the covariance matrix
    chol_cov_matrix = np.linalg.cholesky(sample_covariance_matrix)
    # Generate ellipse offset based on Cholesky decomposition
    ellipse_offset = np.outer(np.cos(theta), chol_cov_matrix[0, :]) + np.outer(np.sin(theta), chol_cov_matrix[1, :])
    # Calculate the points of the prediction interval ellipse
    prediction_ellipse_points = sample_mean + radius * ellipse_offset
    return prediction_ellipse_points


## Load data

In [ ]:
# Read in data from hpar optimisation
df_ffnn = pd.read_json(
    '../saved_models/results_ffnn_opt_bigger.jsonl',
    lines=True
)
df_cusum = pd.read_json(
    '../saved_models/results_cusum_opt_bigger.jsonl',
    lines=True
)
df_hmm = pd.read_json(
    '../saved_models/results_hmm_opt_bigger.jsonl',
    lines=True
)
# Concat the dataframes together, and then do a 
# copy to avoid a dataframe fragmentation warning
# Reset the index to avoid a seaborn error https://github.com/mwaskom/seaborn/issues/3291
df = pd.concat((
    df_ffnn, 
    df_cusum, 
    df_hmm
)).reset_index(drop=True).copy()

### Calculate some auxillary values

In [ ]:
# Preprocess the data a little bit, and get a list of dependant variables
# Preprocess the df a bit to get some nice-to-use columns
df['ffnn.nodes_per_layer.1'] = df['ffnn.nodes_per_layer'].apply(
    lambda x: x[0] if isinstance(x, list) else x
)
df['ffnn.nodes_per_layer.2'] = df['ffnn.nodes_per_layer'].apply(
    lambda x: x[1] if isinstance(x, list) else x
)
# Calculate ratios

avgs = ('macro avg', 'weighted avg')
metrics = ('f1-score', 'precision', 'recall')

for avg in avgs:
    for metric in metrics:
        df[f'ratio.{avg}.{metric}'] = df[f'trn.{avg}.{metric}'] / df[f'val.{avg}.{metric}']
        df[f'ratio.{avg}.{metric}'] = np.where(
            np.isfinite(df[f'ratio.{avg}.{metric}']),
            df[f'ratio.{avg}.{metric}'],
            np.nan
        )

# Print out a list of dependant variables
dep_vars = sorted([
    c for c in df.columns 
    if 'val' not in c and 'trn' not in c and 'ratio' not in c and c not in (
        'saved_at', 'fit_time', 'preprocessing.gesture_allowlist', 
)], key=lambda c: str(c))
print(f"Dependant variables: {dep_vars}")
print("\nVariables which change:")
max_len = max(map(lambda x: len(x), dep_vars))
# Print out all dependant variables that change
for var in dep_vars:
    uniq = df[var].apply(lambda x: str(x) if isinstance(x, list) else x).unique()
    if len(uniq) > 1:
        print(f"{var: <{max_len}} {uniq}")
        
df['preprocessing.num_gesture_classes'] = df['preprocessing.num_gesture_classes'].astype(str)
df['ffnn.dropout_rate'] = np.round(df['ffnn.dropout_rate'], 6)

# Plot the data

## FFNN vs HMM vs CuSUM

In [ ]:
# TODO Add violin plot
# TODO look at F1 scores
# TODO look at f1=1.0 for FFNN
# TODO discuss variance differences within hpars/models (in report)

In [ ]:
subset = df
(
    so.Plot(subset, x='model_type', y='val.macro avg.f1-score', color='model_type')
    .layout(size=(8, 6))
    .add(so.Dots(pointsize=3), so.Jitter())
    .facet(row='preprocessing.num_gesture_classes')
    .limit(y=(-0.05, 1.05))
    .label(
        x="Model Type", 
        color='Model Type',
        y="Macro Average\n$F_1$ Score",
        title="{} Gesture Classes".format,
    )
)

## CuSUM Hyperparameter Comparison

In [ ]:
# TODO investigate threshold range <300
# TODO replace dots with envelope graph
# TODO recall shouldn't increase with threshold

In [ ]:
subset = df[df.model_type=='CuSUM']
(
    so.Plot(subset, x='cusum.thresh', y='val.macro avg.f1-score')
    .layout(size=(8, 6))
    .add(so.Dots(pointsize=3), so.Jitter())
    .facet(row='preprocessing.num_gesture_classes')
    .limit(y=(-.1, 1.1))
    .label(
        x="CuSUM Threshold", 
#         color='Model Type',
#         y="Macro Average\n$F_1$ Score",
        title="CuSUM models\n({} Gesture Classes)".format,
    )
)

## HMM Comparison

In [ ]:
# TODO violin plot here
# TODO remove space in the middle

In [ ]:
subset = df[df.model_type=='HMM']
(
    so.Plot(subset, x='preprocessing.num_gesture_classes', y='val.macro avg.f1-score')
#     .layout(size=(8, 6))
    .add(so.Dots(pointsize=3), so.Jitter())
    .limit(y=(-.1, 1.1))
    .label(
        x="Number of Gesture Classes", 
        y="Macro Average\n$F_1$ Score",
        title="HMM models",
    )
)

## FFNN Hyperparameter Comparison

In [ ]:
# TODO plot f1 against regularization metric & fit gaussian & color=regularization type

In [ ]:
cols = 'ffnn.dropout_rate'
cols_name = 'Dropout'
rows = 'ffnn.l2_coefficient'
rows_name = 'L2'

y = 'val.macro avg.f1-score'
y_name = 'Val. F1'
x = 'ratio.macro avg.f1-score'
x_name = 'Ratio F1'

colors = 'preprocessing.num_gesture_classes'
color_name = '#Classes'

data = df
data = data.dropna(subset=[x, y, colors])

fig, axs = plt.subplots(
    data[cols].nunique(),
    data[rows].nunique(),
    figsize=(
        data[rows].nunique() * 3,
        data[cols].nunique() * 3,
    ),
    squeeze=False,
)

xlim = (
    data.loc[np.isfinite(data[x]), x].min() * 0.95, 
    data.loc[np.isfinite(data[x]), x].max() * 1.05
)
ylim = (
    data.loc[np.isfinite(data[y]), y].min() * 0.95, 
    data.loc[np.isfinite(data[y]), y].max() * 1.05
)

for i, col in enumerate(data[cols].unique()):
    for j, row in enumerate(data[rows].unique()):
        
        subset = data[
            (data[cols] == col) & 
            (data[rows] == row)
        ]
        for k, color in enumerate(subset[colors].unique()):
            subsubset = subset[subset[colors] == color]
            axs[i, j].scatter(
                subsubset[x],
                subsubset[y],
                s=5,
                label=color,
                alpha=0.5
            )
            if subsubset[x].nunique() > 1 and subsubset[y].nunique() > 1:
                ellipse = calculate_prediction_ellipse(
                    subsubset[x],
                    subsubset[y],
                )
                axs[i, j].plot(
                    ellipse[:, 0],
                    ellipse[:, 1],
                )
        axs[i, j].set(
            title=f'{cols_name}: {col} | {rows_name}: {row}',
            ylim=ylim,
            xlim=xlim,
        )
        if j == 0:
            axs[i, j].set(
                ylabel=f"{y_name}"
            )       
        if i == data[cols].nunique() - 1:
            axs[i, j].set(
                xlabel=f"{x_name}"
            )

fig.legend(
    bbox_to_anchor=(1.1, 1.05)
)
plt.tight_layout()

In [ ]:
changing_ffnn_vars = [
    'ffnn.dropout_rate',
    'nn.learning_rate',
    'ffnn.l2_coefficient',
    'ffnn.nodes_per_layer.1',
    'ffnn.nodes_per_layer.2',
]

for var_of_interest in changing_ffnn_vars:
    subset = df[df.model_type=='FFNN'].assign(**{
        # Convertnum_gesture_classes to a str so it is treated categorically
        'preprocessing.num_gesture_classes': lambda df: df['preprocessing.num_gesture_classes'].apply(str),
#         var_of_interest: lambda df: df[var_of_interest].apply(lambda x: np.round(x, 6)),
    })
    (
        so.Plot(subset, x='preprocessing.num_gesture_classes')
        .add(so.Dots(pointsize=3), so.Jitter())
        .pair(y=['val.macro avg.f1-score', 'val.macro avg.recall', 'val.macro avg.precision'])
        .facet(col=var_of_interest)
        .limit(y0=(-.05, 1.05), y1=(-.05, 1.05), y2=(-.05, 1.05))
        .label(
            title=lambda s: var_of_interest + ': \n{}'.format(s),
            x="#Gesture Classes",
            y0="Macro Average\n$F_1$ Score",
            y1="Macro Average\nRecall",
            y2="Macro Average\nPrecision",
        )
        .show()
    )

### View Confusion matrix of a model

In [ ]:
best = (
    df[(df['preprocessing.num_gesture_classes'] == '51')]
    .sort_values(['model_type', 'val.macro avg.f1-score'])
    .groupby('model_type')
    .tail()
    .sort_values('val.macro avg.f1-score', ascending=False)
    [['val.macro avg.f1-score', 'model_dir', 'model_type']]
    .groupby('model_type')
)

fig, axs = plt.subplots(5, 3, figsize=(9, 15))

for i, grouping in enumerate(best.groups):
    group = best.get_group(grouping)
    for j, (_, row) in enumerate(group.iterrows()):
        show_conf_mat_from_model(f"../{row['model_dir']}", axs[j, i])
        axs[j, i].set(
            title=f"{row['model_type']} ({np.round(row['val.macro avg.f1-score'], 4)})",
        )
    

In [ ]:
model_dir = '../saved_models/ffnn_2023-08-25T17:47:12.698190'
def get_npz_data_from_model(model_dir):
    data = np.load(f'{model_dir}/y_val_true_y_val_pred.npz')
    y_true = data['y_true']
    y_pred = data['y_pred']
    return y_true, y_pred

def show_conf_mat_from_model(model_dir, ax=None):
    y_true, y_pred = get_npz_data_from_model(model_dir)
    cm_val = tf.math.confusion_matrix(
        y_true.flatten(), 
        y_pred.flatten()
    ).numpy()
    cm_val[-1, -1] = 0
    return sns.heatmap(
        cm_val,
        annot=False,
        fmt='d',
        square=True,
        mask=(cm_val==0),
        cmap='viridis',
        ax=ax,
    )



show_conf_mat_from_model(model_dir)

In [ ]:
df.loc[
    df['val.macro avg.f1-score'] > 0.9,
    dep_vars + [c for c in df.columns if 'avg' in c and 'val' in c]
]

In [ ]:
def f1(p, r):
    return 2 * (p*r)/(p+r)

f1(0.019608, 0.977864)

# Visualise mis-predictions

Question: Why are the FFNNs mis-predicting so many of the gestures?

In [ ]:
# Load in the dataset/classifier
(
    X_trn, X_val, y_trn, y_val, dt_trn, dt_val
) = common.read_and_split_from_npz("../gesture_data/trn_20.npz")
clf = models.load_tf('../saved_models/ffnn_f1=.2341')
const = common.read_constants('../src/constants.yaml')
sensor_names = list(const['sensors'].values())

In [ ]:
df = read.read_data(
    "../gesture_data/train/",
    offsets_path='../offsets.csv',
    constants_path="../src/constants.yaml"
).sort_values('datetime')
X = df[sensor_names].values
y = df['gesture'].values
dt = df['datetime'].values

In [ ]:
true_label = 1
pred_label = 50
y_pred = clf.predict(X_val)
subset_mask = (pred_label == y_pred) & (true_label == y_val)
idx = np.nonzero(subset_mask)[0][0]

In [ ]:
print(dt_val[idx], y_val[idx], X_val[idx, -1, :])
plt.plot(X_val[idx, :, :])
plt.show()

In [ ]:
origin = dt_val[idx]
delta = pd.Timedelta(1., 'seconds')
sub = df.loc[
    df['datetime'].between(
        origin - delta,
        origin + delta,
    )
]

minus_delta_to_origin = np.nonzero(df['datetime'].between(
    origin - delta,
    origin,
))[0].shape[0]

cmp_ts(
    [sub[sensor_names].values], 
    span=(max(0, minus_delta_to_origin - 20), minus_delta_to_origin)
)

#### A function to compare time series

In [ ]:
def cmp_ts(time_series, span=None):
    """Compare a list of time series datasets.
    Each of the time series in `time_series` must have the same shape,
    and that shape must be (_, 30).
    """
    assert all(ts.shape == time_series[0].shape for ts in time_series)
    assert all(len(ts.shape) == 2 for ts in time_series)
    assert all(ts.shape[1] == 30 for ts in time_series)
    const = common.read_constants('../src/constants.yaml')
    fig, axs = plt.subplots(
        5,
        6,
        figsize=(10, 9)
    )
    ylim = (
        np.array(time_series).min() * 0.9,
        np.array(time_series).max() * 1.1
    )

    for i, ax in enumerate(axs.flatten()):
        ax.set(
            ylim=ylim,
        )
        ax.set_title(list(const['sensors'].values())[i], y=1.0, pad=-14)
        
        if span is not None:
            ax.fill_between(
                range(*span),
                ylim[0],
                ylim[1],
                alpha=0.1,
                color='grey'
            )
        if i % 6 != 0: ax.set_yticks([])
        if i < 24: ax.set_xticks([])

        for ts in time_series:
            ax.plot(
                ts[:, i],
                lw=.5,
                c='grey',
                alpha=.5,
            )
    plt.subplots_adjust(wspace=0, hspace=0)
#     plt.tight_layout()

# Visualising the data

## Compare processed training data per gesture

In [ ]:
delay = 10
(
    X_trn, X_val, y_trn, y_val, dt_trn, dt_val
) = common.read_and_split_from_npz(f"../gesture_data/trn_20_{delay}.npz")
@interact(gesture_index=(0, 49))
def fn(gesture_index=0):
    indices = np.nonzero(y_trn == gesture_index)[0]
    cmp_ts(X_trn[indices])
    plt.suptitle(f'{delay} delay, gesture {gesture_index}')

## Compare raw CSV data per gesture

In [ ]:
# NOTE: This only looks at the original data, not the new data for gesture 45..=49
df = read.read_data(
    "../gesture_data/train/",
    offsets_path='../offsets.csv',
    constants_path="../src/constants.yaml"
).sort_values('datetime')
X = df[sensor_names].values
y = df['gesture'].values
dt = df['datetime'].values

@interact(gesture_index=(0, 49))
def fn(gesture_index=0):
    gesture_label = f'gesture{gesture_index:0>4}' if gesture_index != 50 else 'gesture0255'
    idxs = np.nonzero(y == gesture_label)[0]
    preamble = 30
    postamble = 30
    X_0 = np.zeros((idxs.shape[0], preamble + 1 + postamble, 30))
    for i, idx in enumerate(idxs):
        if idx - preamble < 0 or idx + postamble + 1 >= X.shape[0]:
            continue
        X_0[i] = X[idx-preamble: idx + postamble + 1]

    delay = 15
    cmp_ts(
        X_0,
        span=(max(0, preamble - 20) + delay, preamble + delay)
    )


In [ ]:
idxs = np.nonzero(y == 'gesture0001')[0]
preamble = 20
postamble = 10
X_0 = np.zeros((idxs.shape[0], preamble + 1 + postamble, 30))
for i, idx in enumerate(idxs):
    if idx - preamble < 0 or idx + postamble + 1 >= X.shape[0]:
        continue
    X_0[i] = X[idx-preamble: idx + postamble + 1]

cmp_ts(
    X_0
)
